In [1]:
import os
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
from torch.utils.data import random_split

In [3]:
train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomCrop(224, padding=24),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(0.05, 0.05, 0.05, 0.05),
            transforms.ToTensor(),
        ])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dev_datasset = datasets.ImageFolder(root='./data/Sign_Classification/traffic_Data/DATA/',
                                    transform=train_transform)

generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset, test_dataset = random_split(dev_datasset, [0.7, 0.15, 0.15], generator)
val_dataset.transform = test_transform
test_dataset.transform = test_transform

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128,
                                          shuffle=False, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128,
                                          shuffle=True, num_workers=8)

In [5]:
def train_classifer_epoch(net, trainloader,
                optimizer, device):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    loss_func = nn.CrossEntropyLoss()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs.to(device))
        loss = loss_func(outputs, targets)
        loss.backward()
        optimizer.step()
        predicted = outputs.argmax(dim=1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        train_loss += loss.item()
    train_loss = train_loss/(batch_idx+1)
    train_acc = correct/total
    return train_acc, train_loss

def test_classifer_accuracy(net, testloader, device):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    loss_func = nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            targets = targets.to(device)
            outputs = net(inputs.to(device))
            loss = loss_func(outputs, targets)
            test_loss += loss.item()
            predicted = outputs.argmax(dim=1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    test_acc = correct/total
    test_loss = test_loss/(batch_idx+1)
    return test_acc, test_loss

def get_preds_labels(net, testloader, device):
    net.eval()
    preds = []
    true_labels = []
    sigmoid = nn.Sigmoid()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            outputs = net(inputs.to(device))
            predicted = torch.softmax(outputs, dim=1).detach()
            preds.append(predicted)
            true_labels.append(targets)
        preds = torch.cat(preds)
        true_labels = torch.cat(true_labels)
    return preds.cpu().numpy(), true_labels.cpu().numpy()

In [6]:
def train(net, epochs, trainloader, validloader,
          save_path, device):
    best_acc = -np.inf
    test_acc_history = []
    optimizer = optim.AdamW(net.parameters(),
                          lr=5e-5,)#     weight_decay=5e-4
    for epoch in range(epochs):
        train_acc, train_loss  = train_classifer_epoch(net, trainloader, optimizer,
                                                      device)
        test_acc, test_loss = test_classifer_accuracy(net, validloader, device)
        test_acc_history.append(test_acc)
        print(f'epoch ({epoch+1})| Train loss {round(train_loss, 2)}| Train accuracy {round(train_acc, 2)}| Test accuracy {round(test_acc, 2)}| Test loss {round(test_loss, 2)}')
        if best_acc < test_acc:
            print('Saving model...')
            model_state = {'net': net.state_dict(),
                           'acc': test_acc,
                           'opti': optimizer.state_dict(),
                           'epoch': epoch,
                           'test_acc_history':test_acc_history}
            torch.save(model_state, save_path)
            best_acc = test_acc

In [7]:
# torchvision.models.list_models()

In [8]:

def modify_model_output_classes(model, num_classes):
    if hasattr(model, 'fc'):
        # Common case for models like ResNet
        in_features = model.fc.in_features
        new_classifier = nn.Linear(in_features, num_classes).to(device)
        model.fc = new_classifier
        return model
    
    elif hasattr(model, 'classifier'):
        if isinstance(model.classifier, nn.Linear):
            # Common case for models like densenet, vgg
            in_features = model.classifier.in_features
            new_classifier = nn.Linear(in_features, num_classes).to(device)
            model.classifier = new_classifier
            return model
        
        if isinstance(model.classifier, nn.Sequential) and isinstance(model.classifier[1], nn.Linear):
            # Common case for models like efficientnet_b2
            in_features = model.classifier[1].in_features
            new_classifier = nn.Linear(in_features, num_classes).to(device)
            model.classifier[1] = new_classifier
            return model
    elif hasattr(model, 'heads'):
        # ViT Special case for models with 'heads' as the final layer
        in_features = model.heads.head.in_features
        new_classifier = nn.Linear(in_features, num_classes).to(device)
        model.heads.head = new_classifier
        return model
    elif hasattr(model, 'head'):
        # Swin
        in_features = model.head.in_features
        new_classifier = nn.Linear(in_features, num_classes).to(device)
        model.head = new_classifier
        return model
    else:
        raise ValueError("Unsupported model architecture. Cannot modify output classes.")


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seeds = list(range(20))
seed_counter = 0

for model_type in ['resnet18' ,  'vit_b_16', 'swin_s',  'efficientnet_b2', 'densenet121']:
    for i in range(4):
        seed = seeds[seed_counter]
        torch.random.manual_seed(seed)
        model = torch.hub.load("pytorch/vision", model_type,
                               weights="IMAGENET1K_V1").to(device)
        model = modify_model_output_classes(model, 58)
        train(model, 10, train_loader, val_loader,
              f'./models/road_sign/{model_type}_{i}.ckpt', device)
        seed_counter+=1


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


epoch (1)| Train loss 2.77| Train accuracy 0.39| Test accuracy 0.64| Test loss 1.62
Saving model...
epoch (2)| Train loss 1.17| Train accuracy 0.77| Test accuracy 0.81| Test loss 0.86
Saving model...
epoch (3)| Train loss 0.66| Train accuracy 0.88| Test accuracy 0.9| Test loss 0.55
Saving model...
epoch (4)| Train loss 0.4| Train accuracy 0.94| Test accuracy 0.94| Test loss 0.35
Saving model...
epoch (5)| Train loss 0.25| Train accuracy 0.97| Test accuracy 0.96| Test loss 0.23
Saving model...
epoch (6)| Train loss 0.17| Train accuracy 0.98| Test accuracy 0.97| Test loss 0.17
Saving model...
epoch (7)| Train loss 0.12| Train accuracy 0.99| Test accuracy 0.97| Test loss 0.14
Saving model...
epoch (8)| Train loss 0.09| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.11
Saving model...
epoch (9)| Train loss 0.07| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.1
epoch (10)| Train loss 0.05| Train accuracy 1.0| Test accuracy 0.99| Test loss 0.08
Saving model...


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


epoch (1)| Train loss 2.67| Train accuracy 0.46| Test accuracy 0.64| Test loss 1.6
Saving model...
epoch (2)| Train loss 1.16| Train accuracy 0.8| Test accuracy 0.82| Test loss 0.86
Saving model...
epoch (3)| Train loss 0.64| Train accuracy 0.89| Test accuracy 0.92| Test loss 0.52
Saving model...
epoch (4)| Train loss 0.38| Train accuracy 0.95| Test accuracy 0.95| Test loss 0.31
Saving model...
epoch (5)| Train loss 0.24| Train accuracy 0.97| Test accuracy 0.97| Test loss 0.21
Saving model...
epoch (6)| Train loss 0.16| Train accuracy 0.98| Test accuracy 0.97| Test loss 0.16
Saving model...
epoch (7)| Train loss 0.11| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.13
Saving model...
epoch (8)| Train loss 0.08| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.1
epoch (9)| Train loss 0.06| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.09
Saving model...
epoch (10)| Train loss 0.05| Train accuracy 1.0| Test accuracy 0.98| Test loss 0.08


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


...

epoch (1)| Train loss 3.16| Train accuracy 0.37| Test accuracy 0.65| Test loss 2.5
Saving model...
epoch (2)| Train loss 1.73| Train accuracy 0.77| Test accuracy 0.79| Test loss 1.4
Saving model...
epoch (3)| Train loss 1.03| Train accuracy 0.85| Test accuracy 0.87| Test loss 0.86
Saving model...
epoch (4)| Train loss 0.67| Train accuracy 0.91| Test accuracy 0.92| Test loss 0.61
Saving model...
epoch (5)| Train loss 0.47| Train accuracy 0.94| Test accuracy 0.95| Test loss 0.42
Saving model...
epoch (6)| Train loss 0.33| Train accuracy 0.96| Test accuracy 0.95| Test loss 0.31
Saving model...
epoch (7)| Train loss 0.25| Train accuracy 0.98| Test accuracy 0.97| Test loss 0.24
Saving model...
epoch (8)| Train loss 0.19| Train accuracy 0.98| Test accuracy 0.97| Test loss 0.19
epoch (9)| Train loss 0.14| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.14
Saving model...
epoch (10)| Train loss 0.11| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.12
Saving model...


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


epoch (1)| Train loss 3.09| Train accuracy 0.4| Test accuracy 0.57| Test loss 2.47
Saving model...
epoch (2)| Train loss 1.68| Train accuracy 0.74| Test accuracy 0.79| Test loss 1.4
Saving model...
epoch (3)| Train loss 1.04| Train accuracy 0.84| Test accuracy 0.86| Test loss 0.91
Saving model...
epoch (4)| Train loss 0.7| Train accuracy 0.89| Test accuracy 0.9| Test loss 0.63
Saving model...
epoch (5)| Train loss 0.5| Train accuracy 0.93| Test accuracy 0.93| Test loss 0.47
Saving model...
epoch (6)| Train loss 0.36| Train accuracy 0.96| Test accuracy 0.94| Test loss 0.35
Saving model...
epoch (7)| Train loss 0.27| Train accuracy 0.97| Test accuracy 0.97| Test loss 0.27
Saving model...
epoch (8)| Train loss 0.2| Train accuracy 0.98| Test accuracy 0.98| Test loss 0.21
Saving model...
epoch (9)| Train loss 0.15| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.15
Saving model...
epoch (10)| Train loss 0.12| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.13
Saving model...


Using cache found in /home/guy5/.cache/torch/hub/pytorch_vision_main


epoch (1)| Train loss 3.2| Train accuracy 0.38| Test accuracy 0.63| Test loss 2.49
Saving model...
epoch (2)| Train loss 1.73| Train accuracy 0.74| Test accuracy 0.73| Test loss 1.42
Saving model...
epoch (3)| Train loss 1.06| Train accuracy 0.82| Test accuracy 0.86| Test loss 0.92
Saving model...
epoch (4)| Train loss 0.71| Train accuracy 0.9| Test accuracy 0.91| Test loss 0.63
Saving model...
epoch (5)| Train loss 0.49| Train accuracy 0.93| Test accuracy 0.94| Test loss 0.44
Saving model...
epoch (6)| Train loss 0.36| Train accuracy 0.95| Test accuracy 0.95| Test loss 0.34
Saving model...
epoch (7)| Train loss 0.27| Train accuracy 0.97| Test accuracy 0.97| Test loss 0.25
Saving model...
epoch (8)| Train loss 0.2| Train accuracy 0.98| Test accuracy 0.98| Test loss 0.2
Saving model...
epoch (9)| Train loss 0.16| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.16
Saving model...
epoch (10)| Train loss 0.12| Train accuracy 0.99| Test accuracy 0.98| Test loss 0.13


In [10]:
data, targets  = next(iter(test_loader))

In [11]:
model.eval()
preds = model(data.to(device)).argmax(dim=1)

In [12]:
(targets == preds.cpu())

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True])

In [13]:
net.load_state_dict(torch.load('./models/x_ray_victim.ckpt')['net'])

NameError: name 'net' is not defined

In [ ]:
preds, labels = get_preds_labels(net, test_loader, device)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

def roc_curve_ood(preds, labels):
    fpr, tpr, thresholds = roc_curve(labels, preds)
    thres_idx = np.argmin(np.square(tpr-0.95))
    thres95 = thresholds[thres_idx]
    auc_score = roc_auc_score(labels, preds)
    return thres95, auc_score


thres95, auc = roc_curve_ood(preds, labels)
print(f'AUROC: {auc}')

### Adversarial attack(un-adaptive)

In [ ]:
def GradientPGDAttack(model, inputs, labels,
                                   loss_func, eps, device):
    pertubation = torch.zeros(inputs.size()).to(device)
    samples = inputs.to(device)
    for i in range(50):
        pertubation = torch.autograd.Variable(pertubation, requires_grad=True)
        classifer_outputs = model((samples+pertubation))
        classifier_labels = labels.to(device)
        loss = loss_func(classifer_outputs, classifier_labels)
        loss.backward()
        gradient = torch.ge(pertubation.grad.data, 0)
        pertubation = torch.add(pertubation.data, +(eps/10)*gradient).detach()    
        pertubation = torch.clamp(pertubation, min = -eps, max= eps)
        del classifer_outputs, classifier_labels, gradient, loss
    return (samples+pertubation)

def gen_adv_dataset(model, loader, eps):
    data = []
    labels = []
    for samples, targets in loader:
        labels.append(targets)
        samples, targets = samples.to(device), targets.to(device)

        pertubed = GradientPGDAttack(model, samples, targets,
                                  BinaryCrossEntropy(),
                                  eps, device).detach().cpu()
        data.append(pertubed)
    data = torch.cat(data)
    labels = torch.cat(labels)
    dataset = torch.utils.data.TensorDataset(data, labels)
    return torch.utils.data.DataLoader(dataset, batch_size=128,
                      shuffle=False,num_workers =6,pin_memory = True)

In [ ]:
adv_dataloader = gen_adv_dataset(net, test_loader, 8/255)

In [ ]:
preds, labels = get_preds_labels(net, adv_dataloader, device)
thres95, auc = roc_curve_ood(preds, labels)
print(f'AUROC: {auc}')

# MGM

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [ ]:
def net_pen_rep(net, x):
    out = net.conv1(x)
    out = net.bn1(out)
    out = net.relu(out)
    out = net.layer1(out)
    out = net.layer2(out)
    out = net.layer3(out)
    out = net.layer4(out)
    out = net.avgpool(out)
    return out

def get_pen_reps(net, loader, device):
    net.eval()
    reps = []
    with torch.no_grad():
        for (inputs, targets) in loader:
            inputs= inputs.to(device)
            input_reps = net_pen_rep(net, inputs)
            input_reps = input_reps.detach().cpu()
            reps.append(input_reps)
        reps = torch.cat(reps).squeeze()
        return reps    
    
def calc_MGM_params(net, trainloader):
    train_reps = get_pen_reps(net,
                              trainloader,
                              device).t()
    m = train_reps.size(1) 
    f_dim = train_reps.size(0)
    mu = train_reps.mean(dim = 1,  keepdim=True)
    train_reps -= mu
    cov = (1/ (m+1)) * train_reps.matmul(train_reps.t()) +1e-10*torch.eye(f_dim, f_dim)
    R = torch.cholesky(cov, upper=False)
    R_diag_sum = R.diag().sum()
    R_inv = torch.cholesky_inverse(R)
    return f_dim, R_diag_sum, R_inv, mu
    
def calc_likelihood(rep, f_dim, R_diag_sum, R_inv, mu):
    Z = -0.5*R_diag_sum -f_dim*np.log(2*np.pi)
    log_exp = (R_inv.matmul(rep - mu)**2).sum(dim=0)
    return Z+log_exp
    
def predict_liklihood(net, loader,f_dim, R_diag_sum, R_inv, mu):
    net.eval()
    preds = []
    with torch.no_grad():
        for (inputs, targets) in loader:
            inputs= inputs.to(device)
            reps = net_pen_rep(net, inputs)
            reps = reps.detach().cpu()
            pred = calc_likelihood(reps.squeeze().t(), f_dim, 
                                   R_diag_sum, R_inv, mu)
            preds.append(pred)
        preds = torch.cat(preds)
    return preds

In [ ]:
f_dim, R_diag_sum, R_inv, mu = calc_MGM_params(net, train_loader)

In [ ]:
adv_likelihood = predict_liklihood(net, adv_dataloader, f_dim, R_diag_sum, R_inv, mu)
bengin_likelihood = predict_liklihood(net, test_loader, f_dim, R_diag_sum, R_inv, mu)

In [ ]:
plt.hist(adv_likelihood.numpy(), bins=100, color='r')
plt.hist(bengin_likelihood.numpy(), bins=100, color='b')
plt.show()